In [35]:
# Mrityunjay Kumar #
# RNN Classifiers  #

In [36]:
import tensorflow as tf
import input_data

In [37]:
tf.compat.v1.set_random_seed(1)

In [38]:
mnist = input_data.read_data_sets('MNIST/',one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [39]:
# Mrityunjay kumar #
# CNN Convolution Networks #

tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()


In [40]:
#hyperparameters
lr = 0.001 #learning rate
training_iters = 100000 #number of training iterations
batch_size = 128

n_inputs = 28   # MNIST data input (img shape: 28*28)
n_steps = 28    # time steps (number of time layers)
n_hidden_units = 128   # neurons in hidden layer
n_classes = 10      # MNIST classes (0-9 digits)

In [41]:
# tf Graph input
x = tf.compat.v1.placeholder(tf.compat.v1.float32, [None, n_steps, n_inputs])
y = tf.compat.v1.placeholder(tf.compat.v1.float32, [None, n_classes])

# Define weights
weights = {
    # (28, 128)
    'in': tf.compat.v1.Variable(tf.compat.v1.random_normal([n_inputs, n_hidden_units])),
    # (128, 10)
    'out': tf.compat.v1.Variable(tf.compat.v1.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # (128, )
    'in': tf.compat.v1.Variable(tf.compat.v1.constant(0.1, shape=[n_hidden_units, ])),
    # (10, )
    'out': tf.compat.v1.Variable(tf.compat.v1.constant(0.1, shape=[n_classes, ]))
}

In [42]:
def RNN(X,weights,biases):
    # hidden layer for input to cell
    ########################################

    # transpose the inputs shape from
    # X ==> (128 batch * 28 steps, 28 inputs)
    X = tf.compat.v1.reshape(X,[-1,n_steps])
    
    # into hidden
    # X_in = (128 batch * 28 steps, 128 hidden)
    X_in = tf.compat.v1.matmul(X,weights['in'])+biases['in']
    # X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.compat.v1.reshape(X_in,[-1,n_steps,n_hidden_units])
    
    # cell
    ##########################################

    # basic LSTM Cell.
    lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1,state_is_tuple=True)
    # lstm cell is divided into two parts (c_state, h_state)
    _init_state = lstm_cell.zero_state(batch_size,dtype=tf.float32)
    
    # You have 2 options for following step.
    # 1: tf.nn.rnn(cell, inputs);
    # 2: tf.nn.dynamic_rnn(cell, inputs).
    # If use option 1, you have to modified the shape of X_in, go and check out this:
    # https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py
    # In here, we go for option 2.
    # dynamic_rnn receive Tensor (batch, steps, inputs) or (steps, batch, inputs) as X_in.
    # Make sure the time_major is changed accordingly.
    # time major is used to put the 1st cell of the tensor shape received by dynamic_rnn as time_steps or not
    # here it's [batch_size,steps,inputs]  => so time_major is False.
    
    outputs, final_state = tf.compat.v1.nn.dynamic_rnn(lstm_cell,initial_state=_init_state,inputs=X_in,time_major=False)
    
    # hidden layer for output as the final results
    #############################################
    # results = tf.matmul(final_state[1], weights['out']) + biases['out']

    # # or
    # unpack to list [(batch, outputs)..] * steps
    outputs = tf.compat.v1.unstack(tf.transpose(outputs, [1, 0, 2]))    # states is the last outputs
    results = tf.compat.v1.matmul(outputs[-1], weights['out']) + biases['out']

    return results

In [43]:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
train_op = tf.compat.v1.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.compat.v1.equal(tf.compat.v1.argmax(pred, 1), tf.compat.v1.argmax(y, 1))
accuracy = tf.compat.v1.reduce_mean(tf.compat.v1.cast(correct_pred, tf.compat.v1.float32))

init = tf.compat.v1.initialize_all_variables()

E0129 11:45:43.572850 4445218240 tf_should_use.py:71] ==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'init' type=NoOp>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()  File "/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()  File "/anacond

In [44]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
        }))
        step += 1

0.09375
0.125
0.0703125
0.125
0.0390625
0.125
0.125
0.0625
0.0703125
0.125
0.09375
0.109375
0.09375
0.1171875
0.140625
0.09375
0.09375
0.0859375
0.0703125
0.078125
0.0546875
0.0625
0.0625
0.078125
0.1484375
0.125
0.0859375
0.03125
0.09375
0.109375
0.109375
0.1171875
0.0859375
0.1484375
0.15625
0.1171875
0.109375
0.078125
0.0859375
0.125
